# multiple regions intersect

In [1]:
# step1 load reference order

ref_order_dict = {}

ref_fai = open("./hg38_only_chromosome.fa.fai", "r")

index = 0 
for line in ref_fai:
    line_list = line.strip().split("\t")
    ref_order_dict[line_list[0]] = index
    index += 1 
    
ref_fai.close()
        
ref_order_dict

{'chr1': 0,
 'chr2': 1,
 'chr3': 2,
 'chr4': 3,
 'chr5': 4,
 'chr6': 5,
 'chr7': 6,
 'chr8': 7,
 'chr9': 8,
 'chr10': 9,
 'chr11': 10,
 'chr12': 11,
 'chr13': 12,
 'chr14': 13,
 'chr15': 14,
 'chr16': 15,
 'chr17': 16,
 'chr18': 17,
 'chr19': 18,
 'chr20': 19,
 'chr21': 20,
 'chr22': 21,
 'chrX': 22,
 'chrY': 23,
 'chrM': 24}

In [2]:
def cmp_region(region_a, region_b, ref_order_dict):
    """
    INPUT:
        <region_a> <region_b>
            str, line from BED file
        
    OUTPUT:
        -1  region_a at upstream of region_b
        0   region_a overlaps with region_b
        1   region_a at downstream of region_b
    """
    if region_a is None:
        return 1
    
    if region_b is None:
        return -1 
        
    region_list_a = region_a.strip().split("\t")
    region_list_b = region_b.strip().split("\t")
    
    order_a = ref_order_dict.get(region_list_a[0])
    order_b = ref_order_dict.get(region_list_b[0])
    
    if order_a < order_b:
        return -1 
    
    if order_a > order_b:
        return 1 
    
    # get region int
    start_a = int(region_list_a[1])
    end_a = int(region_list_a[2])

    start_b = int(region_list_b[1])
    end_b = int(region_list_b[2])    
        
    # no-onverlap a upstream
    if end_a < start_b:
        return -1 

    # no-overlap a downstream
    elif start_a > end_b:
        return 1
        
    return 0 
 
    

In [3]:
def update_temp_info(temp_merge_line, temp_chr_name, temp_chr_start, temp_chr_end, new_line):
    """
    """
    
    new_line_list = new_line.strip().split("\t")
    
    if temp_merge_line is None:
        temp_merge_line = new_line
        temp_chr_name = new_line_list[0]
        temp_chr_start = int(new_line_list[1])
        temp_chr_end = int(new_line_list[2])

    else:
        temp_chr_name = new_line_list[0]
        temp_chr_start = min(temp_chr_start, int(new_line_list[1]))
        temp_chr_end = max(temp_chr_end, int(new_line_list[2]))
        temp_merge_line = "%s\t%s\t%s" % (temp_chr_name, temp_chr_start, temp_chr_end)   
    
    return temp_merge_line, temp_chr_name, temp_chr_start, temp_chr_end

In [4]:
# find all a-b intersect info

# 2.1 open file 
bed_a = open("./test_multi_region_a.sort.bed", "r")
bed_b = open("./test_multi_region_b.sort.bed", "r")

# 2.2 init
line_a = bed_a.readline()
line_b = bed_b.readline()

temp_list = []
temp_chr_name = None
temp_chr_start = None
temp_chr_end = None
temp_merge_line = None
new_line_state = True

while line_a and line_b:
    
    # check temp list
    if len(temp_list) > 0 and new_line_state: 

        temp_cmp_res = cmp_region(line_a, temp_merge_line, ref_order_dict)
        
        if temp_cmp_res == 1:
            temp_list = []
            temp_chr_name = None
            temp_chr_start = None
            temp_chr_end = None
            temp_merge_line = None

        elif temp_cmp_res == 0:
            for temp_line_b in temp_list:
                run_cmp_res = cmp_region(line_a, temp_line_b, ref_order_dict)
                
                if run_cmp_res == 0:
                    print(line_a.strip(), temp_line_b.strip())
                    print("-" * 80)

    # check current line overlap
    cmp_res = cmp_region(line_a, line_b, ref_order_dict)
    
    if cmp_res == -1:        
        line_a = bed_a.readline()
        new_line_state = True
    
    elif cmp_res == 1:
        new_line_state = False
        
        line_b_list = line_b.strip().split("\t")
        
        if line_a.split("\t")[0] == line_b_list[0]:   
            temp_list.append(line_b)
            
            temp_merge_line, temp_chr_name, temp_chr_start, temp_chr_end = update_temp_info(
                temp_merge_line, 
                temp_chr_name, 
                temp_chr_start, 
                temp_chr_end,
                line_b
            )          
            
        else:
            temp_list = []
            temp_chr_name = None
            temp_chr_start = None
            temp_chr_end = None
            temp_merge_line = None
            
        line_b = bed_b.readline()
    
    else:
        new_line_state = False
        print(line_a.strip(), line_b.strip())
        print("-" * 80)

        temp_list.append(line_b)
        temp_merge_line, temp_chr_name, temp_chr_start, temp_chr_end = update_temp_info(
                temp_merge_line, 
                temp_chr_name, 
                temp_chr_start, 
                temp_chr_end,
                line_b
        )  
        
        line_b = bed_b.readline()
        
# check file end
file_b_end_state = False
file_a_end_state = False

if line_b == "":
    file_b_end_state = True

if line_a == "":
    file_a_end_state = True
    
while not file_a_end_state:
    
    # read new line
    line_a = bed_a.readline()
    new_line_state = True
    
    if line_a == "":
        break

    # check temp list
    if len(temp_list) > 0 and new_line_state: 

        temp_cmp_res = cmp_region(line_a, temp_merge_line, ref_order_dict)
        
        if temp_cmp_res == 1:
            break

        elif temp_cmp_res == 0:
            for temp_line_b in temp_list:
                run_cmp_res = cmp_region(line_a, temp_line_b, ref_order_dict)
                
                if run_cmp_res == 0:
                    print(line_a.strip(), temp_line_b.strip())
                    print("-" * 80)
    

    

        

chr1	140	170	a1 chr1	100	150	b1
--------------------------------------------------------------------------------
chr1	140	170	a1 chr1	120	180	b2
--------------------------------------------------------------------------------
chr1	140	170	a1 chr1	130	200	b3
--------------------------------------------------------------------------------
chr1	145	280	a2 chr1	100	150	b1
--------------------------------------------------------------------------------
chr1	145	280	a2 chr1	120	180	b2
--------------------------------------------------------------------------------
chr1	145	280	a2 chr1	130	200	b3
--------------------------------------------------------------------------------
chr1	145	280	a2 chr1	175	300	b4
--------------------------------------------------------------------------------
chr1	145	280	a2 chr1	260	400	b5
--------------------------------------------------------------------------------
chr1	290	350	a3 chr1	175	300	b4
----------------------------------------------------------------